In [ ]:
#

In [ ]:
#pip install contractions
#pip install fasttext
#pip install tqdm
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import fasttext
import contractions
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [ ]:
#test = pd.read_csv('Howard-Local-NOTEEVENTS.csv.gz',nrows=5000)
from tqdm import tqdm
chunksize = 50000
count = 0
fields = ['CATEGORY','TEXT']
listOFChunks = []
for chunk in pd.read_csv("Howard-Local-NOTEEVENTS.csv.gz", chunksize=chunksize,usecols=fields):
    #test.append(chunk)
    for report in tqdm(range(len(chunk['TEXT']))):
        try:
            chunk['TEXT'][report+count] = chunk['TEXT'][report+count].replace("[**", "").replace("**]", "")
        except:
            None
        #test['TEXT'] = test['TEXT'].replace("[**", "").replace("**]", "")
    count += chunksize
    listOFChunks.append(chunk)
    
test = pd.concat(listOFChunks)
#test = pd.read_csv('s3://howardvip-clinical-data/NOTEEVENTS.csv.gz')

In [ ]:
"""
from tqdm import tqdm
for report in tqdm(range(len(test['TEXT']))):
    test['TEXT'][report] = test['TEXT'][report].replace("[**", "").replace("**]", "")
"""

In [ ]:
"""
test.drop('ROW_ID', axis=1, inplace=True)
test.drop('SUBJECT_ID', axis=1, inplace=True)
test.drop('HADM_ID', axis=1, inplace=True)
test.drop('CHARTDATE', axis=1, inplace=True)
test.drop('STORETIME', axis=1, inplace=True)
test.drop('DESCRIPTION', axis=1, inplace=True)
test.drop('CGID', axis=1, inplace=True)
test.drop('ISERROR', axis=1, inplace=True)
test.drop('CHARTTIME', axis=1, inplace=True)
"""
textClean = test.loc[:, ['CATEGORY', 'TEXT']]


In [ ]:
textClean

In [ ]:
for col in test.columns:
    print(col, test[col].isnull().sum())

In [ ]:
textClean['text_report'] = textClean['TEXT'].apply(lambda x: [contractions.fix(word) for word in x.split()])
textClean.head()

In [ ]:
textClean['text_report_str'] = [' '.join(map(str, l)) for l in textClean['text_report']]
textClean.head()

In [ ]:
nltk.download('punkt')

In [ ]:
textClean['tokenized'] = textClean['text_report_str'].apply(word_tokenize)
textClean.head()

In [ ]:
textClean['lower'] = textClean['tokenized'].apply(lambda x: [word.lower() for word in x])
textClean.head()

In [ ]:
punc = string.punctuation
textClean['no_punc'] = textClean['lower'].apply(lambda x: [word for word in x if word not in punc])
textClean.head()

In [ ]:
nltk.download('wordnet')

In [ ]:
stop_words = set(stopwords.words('english'))
textClean['stopwords_removed'] = textClean['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
textClean.head()

In [ ]:
textClean['pos_tags'] = textClean['stopwords_removed'].apply(nltk.tag.pos_tag)
textClean.head()

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
textClean['wordnet_pos'] = textClean['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
textClean.head()

In [ ]:
wnl = WordNetLemmatizer()
textClean['lemmatized'] = textClean['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
textClean.head()

In [ ]:
print(textClean['lemmatized'][1])

In [ ]:
textClean.to_csv('ALL_Brackets_notebooks_events_scrape_all_clean.csv')


In [ ]:
import s3fs

bytes_to_write = df.to_csv(None).encode()
fs = s3fs.S3FileSystem(key=key, secret=secret)
with fs.open('s3://howardp-cleaned-parsed-data/ALL_Brackets_notebooks_events_scrape_all_clean.csv', 'wb') as f:
    f.write(bytes_to_write)